In [2]:
import numpy as np
import pandas as pd
import string
import re

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# with open('/content/drive/MyDrive/Scanner/prepare/label-finish_utf8.txt',mode='r',encoding='utf8',errors='ignore') as f:
#     text = f.read()
with open('label-finish_utf8.txt',mode='r',encoding='utf8',errors='ignore') as f:
    text = f.read()

In [5]:
data = list(map(lambda x:x.split('\t'),text.split('\n')))

In [6]:
data

[['id', 'text', 'correctness', 'Label'],
 ['print_1.jpg', '#ี]ริ', '0.237090427', 'O'],
 ['print_1.jpg', 'manpuku', '0.712725311', 'O'],
 ['print_1.jpg', ' manpuku', '0.746942042', 'O'],
 ['print_1.jpg', 'โทรศพท:0624344538', '0.7257566', 'O'],
 ['print_1.jpg', 'ปล', '0.981482155', 'O'],
 ['print_1.jpg', ' โต๊ะ - 13', '0.257888542', 'O'],
 ['print_1.jpg', 'ทานที่ร้าน:', '0.912334081', 'O'],
 ['print_1.jpg', 'ชื่อพนักงาน:', '0.895159894', 'O'],
 ['print_1.jpg', '29-05-2566 19:25', '0.995708661', 'O'],
 ['print_1.jpg', '29-05-2566', '0.998419343', 'O'],
 ['print_1.jpg', 'เวลาทีฟิมฟ์:', '0.926684602', 'O'],
 ['print_1.jpg', 'oty', '0.56886816', 'O'],
 ['print_1.jpg', 'ราคา', '0.998467088', 'O'],
 ['print_1.jpg', 'สินค้า', '0.812716727', 'O'],
 ['print_1.jpg', ' 1', '0.264601721', 'O'],
 ['print_1.jpg', 'ฟันตันเมน (xl)', '0.438113698', 'B-ITEM'],
 ['print_1.jpg', '1', '0.181559522', 'O'],
 ['print_1.jpg', '169.00', '0.999538029', 'O'],
 ['print_1.jpg', '1', '0.873581326', 'O'],
 ['print_1.j

In [7]:
df = pd.DataFrame(data[1:],columns=data[0])

In [8]:
df.head(10)

,id,text,correctness,Label
0,print_1.jpg,#ี]ริ,0.237090427,O
1,print_1.jpg,manpuku,0.712725311,O
2,print_1.jpg,manpuku,0.746942042,O
3,print_1.jpg,โทรศพท:0624344538,0.7257566,O
4,print_1.jpg,ปล,0.981482155,O
5,print_1.jpg,โต๊ะ - 13,0.257888542,O
6,print_1.jpg,ทานที่ร้าน:,0.912334081,O
7,print_1.jpg,ชื่อพนักงาน:,0.895159894,O
8,print_1.jpg,29-05-2566 19:25,0.995708661,O
9,print_1.jpg,29-05-2566,0.998419343,O


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13625 entries, 0 to 13624
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           13625 non-null  object
 1   text         13624 non-null  object
 2   correctness  13624 non-null  object
 3   Label        13624 non-null  object
dtypes: object(4)
memory usage: 425.9+ KB


In [10]:
df['correctness'] = df['correctness'].astype(float)

### Cleaning Text
- Remove white space
- Remove Unwanted special characters

In [11]:
string.whitespace, string.punctuation

(' \t\n\r\x0b\x0c', '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')

In [12]:
# where Label I-DISC', 'I-TOTAL', 'B-COST', 'B-DISC','B-VAT', 'B-TOTAL', 'B-SERVC', 'I-COST', 'I-VAT' let check text
x = df[df['Label'].isin(['I-DISC', 'I-TOTAL', 'B-COST', 'B-DISC','B-VAT', 'B-TOTAL', 'B-SERVC', 'I-COST', 'I-VAT'])]['text'].values
# trim space in front and behind text
split_clean = [i.strip() for i in x] 


clean_number = []
for i in split_clean:
    cn = i
    txt = i.split(' ')
    for index,j in enumerate(txt):
        # if j and j+1 is number then combine it
        if index < len(txt) - 1 and re.match(r'[-0-9,.]*|[-0-9,.]*', j) and re.match(r'[-0-9,.]*|[-0-9,.]*', txt[index + 1]):
            cn = j + txt[index + 1]
            break
        elif re.match(r'[-0-9,.]*|[-0-9,.]*',j):
            cn = j
            break
    clean_number.append(cn)


# if has o in front of number please change it to 0
clean_number = [re.sub(r'[ocCDdq\[\}\(\)a๓กe]','0',i) for i in clean_number]
#if has i or l or ! or | or I or f please change it to 1
clean_number = [re.sub(r'[iI!l|fาเ?tว]','1',i) for i in clean_number]
# if has s or S please change it to 5
clean_number = [re.sub(r'[sS]','5',i) for i in clean_number]
# if has g or q or Q or G please change it to 9
clean_number = [re.sub(r'[gG?]','9',i) for i in clean_number]

# change , to .
clean_number = [re.sub(r',|(\.\s)|(,\s)|(\s,\s)|(\s\.\s)','.',i) for i in clean_number]

# if find more than 1 . please remov it except the last one
clean_number = [re.sub(r'\.(?=.*\.)','',i) for i in clean_number]

# if has - in front of number please remove it
clean_number = [re.sub(r'.\*','',i) for i in clean_number]

# if has * 
clean_number = [re.sub(r'(.*-)|(\')','',i) for i in clean_number]

# if has a char  
clean_number = [re.sub(r'[a-zA-Zก-ฮ:\{\}]','',i) for i in clean_number]

# if in clean_number =^\D. is True then put 00 behide it
# clean_number = [i+'00' if re.match(r'(\d*\.(\W|))',i) else i for i in clean_number]
# if not [0-9] OR dot then change it to 0
clean_number = [re.sub(r'[^\d\.]','0',i) for i in clean_number]

# if '' in clean_number then change it to 0
clean_number = [re.sub(r'\'\'','0',i) for i in clean_number]





In [13]:
clean_number

['22.10',
 '419.9',
 '149.00',
 '169.00',
 '55.00',
 '69.00',
 '135.00',
 '8.8301',
 '135.00',
 '135.00',
 '579.00',
 '315.00',
 '698.00',
 '90.00',
 '180.00',
 '75.00',
 '579.00',
 '175.00',
 '100.00',
 '579.00',
 '3949.00',
 '1198.00',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '119.80',
 '1317.80',
 '86.21',
 '196.00',
 '196.00',
 '392.00',
 '54.88',
 '22.50',
 '816.38',
 '145.00',
 '150010408',
 '0.00',
 '0.00',
 '105.00',
 '105.00',
 '6.8701',
 '74.00',
 '76.00',
 '76.00',
 '226.00',
 '0.00',
 '1533.00',
 '91.59',
 '273.00',
 '273.00',
 '132.83',
 '2030.00',
 '178.00',
 '45.00',
 '229.00',
 '114.50',
 '337.50',
 '337.50',
 '337.50',
 '337.50',
 '43.00',
 '43.00',
 '180.000',
 '11.78',
 '180.00',
 '29.00',
 '20.00',
 '20.00',
 '69.00',
 '19.00',
 '28.00',
 '11.00',
 '55.00',
 '55.00',
 '3.60',
 '360.00',
 '50.00',
 '410.00',
 '2814966.70',
 '3835431.05',
 '100000.00',
 '149526.08',
 '907865.84',
 '6802057.00',
 '250.00',
 '10

In [14]:
# change x and clean_number to dataframe
df_clean = pd.DataFrame({'text':x,'clean_number':clean_number})
df_clean


,text,clean_number
0,22.10,22.10
1,419.9,419.9
2,"149,00",149.00
3,169.00,169.00
4,55.00,55.00
...,...,...
1401,1. 00,1.00
1402,3.00,3.00
1403,1. 00,1.00
1404,1. 00,1.00


In [15]:
# # save to csv
# df_clean.to_csv('clean_number.csv',index=False)

In [286]:
whitespace = string.whitespace
punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
tableWhitespace = str.maketrans('','',whitespace)
tablePunctuation = str.maketrans('','',punctuation)
def cleanText(txt, correctness):
      text = str(txt)
      # text = text.lower()
      removewhitespace = text.translate(tableWhitespace)
      removepunctuation = removewhitespace.translate(tablePunctuation)

      return str(removepunctuation)


In [287]:
df['text'] = df.apply(lambda x: cleanText(x['text'], x['correctness']), axis=1)

In [288]:
dataClean = df.query("text != '' ")
dataClean.dropna(inplace=True)

C:\Users\CPE\AppData\Local\Temp\ipykernel_88476\3379659824.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataClean.dropna(inplace=True)


In [289]:
dataClean.head(10)

,id,text,correctness,Label
0,print_1.jpg,ีริ,0.237090,O
1,print_1.jpg,manpuku,0.712725,O
2,print_1.jpg,manpuku,0.746942,O
3,print_1.jpg,โทรศพท0624344538,0.725757,O
4,print_1.jpg,ปล,0.981482,O
5,print_1.jpg,โต๊ะ-13,0.257889,O
6,print_1.jpg,ทานที่ร้าน,0.912334,O
7,print_1.jpg,ชื่อพนักงาน,0.895160,O
8,print_1.jpg,29-05-25661925,0.995709,O
9,print_1.jpg,29-05-2566,0.998419,O


In [290]:
# drop correctness
dataClean.drop(columns=['correctness'], inplace=True)

C:\Users\CPE\AppData\Local\Temp\ipykernel_88476\2709090372.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataClean.drop(columns=['correctness'], inplace=True)


### Convert Data into Spacy Format

In [291]:
group = dataClean.groupby(by='id')

In [292]:
papers = group.groups.keys()

In [293]:
papers

dict_keys(['print_1.jpg', 'print_10.jpg', 'print_100.jpg', 'print_101.jpg', 'print_102.jpg', 'print_103.jpg', 'print_104.jpg', 'print_105.jpg', 'print_106.jpg', 'print_107.jpg', 'print_108.jpg', 'print_109.jpg', 'print_11.jpg', 'print_110.jpg', 'print_111.jpg', 'print_112.jpg', 'print_113.jpg', 'print_114.jpg', 'print_115.jpg', 'print_116.jpg', 'print_117.jpg', 'print_118.jpg', 'print_119.jpg', 'print_12.jpg', 'print_120.jpg', 'print_121.jpg', 'print_122.jpg', 'print_123.jpg', 'print_124.jpg', 'print_125.jpg', 'print_126.jpg', 'print_127.jpg', 'print_128.jpg', 'print_129.jpg', 'print_13.jpg', 'print_130.jpg', 'print_131.jpg', 'print_132.jpg', 'print_133.jpg', 'print_134.jpg', 'print_135.jpg', 'print_136.jpg', 'print_137.jpg', 'print_138.jpg', 'print_139.jpg', 'print_14.jpg', 'print_140.jpg', 'print_141.jpg', 'print_142.jpg', 'print_143.jpg', 'print_144.jpg', 'print_145.jpg', 'print_146.jpg', 'print_147.jpg', 'print_148.jpg', 'print_149.jpg', 'print_15.jpg', 'print_150.jpg', 'print_151.

In [294]:
group.get_group('print_1.jpg')

,id,text,Label
0,print_1.jpg,ีริ,O
1,print_1.jpg,manpuku,O
2,print_1.jpg,manpuku,O
3,print_1.jpg,โทรศพท0624344538,O
4,print_1.jpg,ปล,O
5,print_1.jpg,โต๊ะ-13,O
6,print_1.jpg,ทานที่ร้าน,O
7,print_1.jpg,ชื่อพนักงาน,O
8,print_1.jpg,29-05-25661925,O
9,print_1.jpg,29-05-2566,O


In [295]:
allreceiptData = []
for paper in papers:
    cardData = []
    grouparray = group.get_group(paper)[['text','Label']].values
    content = ''
    annotations = {'entities':[]}
    start = 0
    end = 0
    for text, label in grouparray:
        text = str(text)
        stringLength = len(text) + 1

        start = end
        end = start + stringLength

        if label != 'O':
            annot = (start,end-1,label)
            annotations['entities'].append(annot)

        content = content + text + ' '


    receiptData = (content,annotations)
    allreceiptData.append(receiptData)

In [296]:
allreceiptData

[('ีริ manpuku manpuku โทรศพท0624344538 ปล โต๊ะ-13 ทานที่ร้าน ชื่อพนักงาน 29-05-25661925 29-05-2566 เวลาทีฟิมฟ์ oty ราคา สินค้า 1 ฟันตันเมนxl 1 169.00 1 2 อุดังเทมประ xl 1 3 ทาโกะบากี od 69.00 69. 4 เต่าปูเปนรานุ ภาคาปกติ a ขอดรวม - 22.10 55 419.9 ฟั้งหมด havaaniee mmeal คุณปริม 1956 เวลาเข้า ราคารวม 149.00 149,00 169.00 55.00 55.00 69.00 69.00 442. ลด ',
  {'entities': [(127, 138, 'B-ITEM'),
    (152, 163, 'B-ITEM'),
    (164, 166, 'I-ITEM'),
    (171, 180, 'B-ITEM'),
    (196, 209, 'B-ITEM'),
    (210, 218, 'I-ITEM'),
    (230, 235, 'I-DISC'),
    (239, 244, 'I-TOTAL'),
    (306, 312, 'B-COST'),
    (313, 319, 'B-COST'),
    (326, 331, 'B-COST'),
    (338, 343, 'B-COST')]}),
 ('starbuckscoffee wiiconftostannuckscoffee 1435cantralaaทa3sthni. 0655043606 navr onandeday 24726210ผaแumon chk0100111998 u21231252p14 hrbiscus oa a3s.0o code payment 135.00 13s.0o 8.83vatttl hetttl 135.00 tax n01c488230721 126.17 tax 6135541008688721incaots5 p0030230002608/ soinstarbucksrewards callect1starfrom

In [297]:
receipt_data_df = pd.DataFrame(allreceiptData,columns=['text','labels'])
receipt_data_df['isNull'] = receipt_data_df['labels'].apply(lambda x: 'Null' if len(x['entities']) ==0
                                                      else 'Clean')

#### Null entries need to drop

In [298]:
receipt_data_df.query('isNull == "Null"')

,text,labels,isNull
5,bangkokbank รี๋ อิษษาบาารกรุเนทบป รุ้ starbuck...,{'entities': []},Null
119,khanorkorsha8u . a40 0010530001547 474 lolooat...,{'entities': []},Null
199,"aanskiksharftaansisie ta32 """"""iiii521212"" aaei...",{'entities': []},Null
264,รุ์ รุ้ ต อี้ bangkukbank อี้ ธหาศารกรุงเทบบ ู...,{'entities': []},Null


#### Consilder only clean data

In [299]:
receipt_data_df.dropna(inplace=True)
clean_data = receipt_data_df.query('isNull == "Clean"')[['text','labels']]

In [300]:
allreceiptData = list(map(lambda x: tuple(x), clean_data.values.tolist()))

In [301]:
allreceiptData

[('ีริ manpuku manpuku โทรศพท0624344538 ปล โต๊ะ-13 ทานที่ร้าน ชื่อพนักงาน 29-05-25661925 29-05-2566 เวลาทีฟิมฟ์ oty ราคา สินค้า 1 ฟันตันเมนxl 1 169.00 1 2 อุดังเทมประ xl 1 3 ทาโกะบากี od 69.00 69. 4 เต่าปูเปนรานุ ภาคาปกติ a ขอดรวม - 22.10 55 419.9 ฟั้งหมด havaaniee mmeal คุณปริม 1956 เวลาเข้า ราคารวม 149.00 149,00 169.00 55.00 55.00 69.00 69.00 442. ลด ',
  {'entities': [(127, 138, 'B-ITEM'),
    (152, 163, 'B-ITEM'),
    (164, 166, 'I-ITEM'),
    (171, 180, 'B-ITEM'),
    (196, 209, 'B-ITEM'),
    (210, 218, 'I-ITEM'),
    (230, 235, 'I-DISC'),
    (239, 244, 'I-TOTAL'),
    (306, 312, 'B-COST'),
    (313, 319, 'B-COST'),
    (326, 331, 'B-COST'),
    (338, 343, 'B-COST')]}),
 ('starbuckscoffee wiiconftostannuckscoffee 1435cantralaaทa3sthni. 0655043606 navr onandeday 24726210ผaแumon chk0100111998 u21231252p14 hrbiscus oa a3s.0o code payment 135.00 13s.0o 8.83vatttl hetttl 135.00 tax n01c488230721 126.17 tax 6135541008688721incaots5 p0030230002608/ soinstarbucksrewards callect1starfrom

## Split the Data into Training and Testing Set

In [302]:
import random

In [303]:
random.shuffle(allreceiptData)

In [304]:
len(allreceiptData)

267

In [305]:
TrainData = allreceiptData[:240]
TestData = allreceiptData[240:]

In [306]:
import pickle

In [308]:
# pickle.dump(TrainData,open('/content/drive/MyDrive/Scanner/prepare/data/TrainData.pickle',mode='wb'))
# pickle.dump(TestData,open('/content/drive/MyDrive/Scanner/prepare/data/TestData.pickle',mode='wb'))

pickle.dump(TrainData,open('data/TrainData.pickle',mode='wb'))
pickle.dump(TestData,open('data/TestData.pickle',mode='wb'))